In [ ]:
chatgpt- GANs and Transformer and diffusion models for market predictions in finance and portfolion management 

python code for Transformer in market prediction 

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout, GlobalAveragePooling1D

# Load and preprocess market data
data = pd.read_csv('market_data.csv')
prices = data['Close'].values.reshape(-1, 1)

scaler = MinMaxScaler()
scaled_prices = scaler.fit_transform(prices)

# Prepare training data
seq_length = 60
X_train, y_train = [], []

for i in range(len(scaled_prices) - seq_length):
    X_train.append(scaled_prices[i:i + seq_length])
    y_train.append(scaled_prices[i + seq_length])

X_train, y_train = np.array(X_train), np.array(y_train)

# Build the Transformer model
def build_transformer_model(seq_length, feature_dim, num_heads=4, ff_dim=64, num_layers=2):
    inputs = Input(shape=(seq_length, feature_dim))
    x = inputs

    for _ in range(num_layers):
        attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=feature_dim)(x, x)
        attn_output = Dropout(0.1)(attn_output)
        x = LayerNormalization(epsilon=1e-6)(x + attn_output)

        ff_output = Dense(ff_dim, activation='relu')(x)
        ff_output = Dense(feature_dim)(ff_output)
        ff_output = Dropout(0.1)(ff_output)
        x = LayerNormalization(epsilon=1e-6)(x + ff_output)

    x = GlobalAveragePooling1D()(x)
    outputs = Dense(feature_dim)(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mse')
    return model


model = build_transformer_model(seq_length, 1)

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32)

# Make predictions
predictions = model.predict(X_train[-1].reshape(1, seq_length, 1))
predicted_price = scaler.inverse_transform(predictions)

print(predicted_price)

# Save the model
model.save('market_prediction_transformer.h5')

# This code builds a Transformer to predict future market prices based on historical data.
# Let me know if you’d like me to fine-tune the architecture, add features, or improve the training process!


python code for Transformer in portfolio management



In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout, GlobalAveragePooling1D

# Load and preprocess data
data = pd.read_csv('portfolio_data.csv')
returns = data.pct_change().dropna()

scaler = MinMaxScaler()
scaled_returns = scaler.fit_transform(returns)

# Prepare training data
seq_length = 60
X_train, y_train = [], []

for i in range(len(scaled_returns) - seq_length):
    X_train.append(scaled_returns[i:i + seq_length])
    y_train.append(scaled_returns[i + seq_length])

X_train, y_train = np.array(X_train), np.array(y_train)

# Build the Transformer model
def build_transformer_model(seq_length, feature_dim, num_heads=4, ff_dim=64, num_layers=2):
    inputs = Input(shape=(seq_length, feature_dim))
    x = inputs

    for _ in range(num_layers):
        attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=feature_dim)(x, x)
        attn_output = Dropout(0.1)(attn_output)
        x = LayerNormalization(epsilon=1e-6)(x + attn_output)

        ff_output = Dense(ff_dim, activation='relu')(x)
        ff_output = Dense(feature_dim)(ff_output)
        ff_output = Dropout(0.1)(ff_output)
        x = LayerNormalization(epsilon=1e-6)(x + ff_output)

    x = GlobalAveragePooling1D()(x)
    outputs = Dense(feature_dim)(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mse')
    return model


model = build_transformer_model(seq_length, returns.shape[1])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32)

# Make predictions
predictions = model.predict(X_train[-1].reshape(1, seq_length, returns.shape[1]))
predicted_returns = scaler.inverse_transform(predictions)

print(predicted_returns)

# Save the model
model.save('portfolio_management_transformer.h5')

# This code sets up a Transformer to analyze historical portfolio returns and predict future returns.
# Let me know if you’d like me to adjust the architecture, add evaluation metrics, or enhance the training process!


python code for GANs in portfolio management



In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, Reshape, LSTM
from sklearn.preprocessing import MinMaxScaler

# Load and preprocess data
data = pd.read_csv('portfolio_data.csv')
returns = data.pct_change().dropna()

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_returns = scaler.fit_transform(returns)

# Prepare training data
seq_length = 60
X_train = []

for i in range(len(scaled_returns) - seq_length):
    X_train.append(scaled_returns[i:i + seq_length])

X_train = np.array(X_train)

# Build the GAN
def build_generator():
    model = Sequential([
        Dense(128, input_dim=100),
        LeakyReLU(alpha=0.2),
        Dense(256),
        LeakyReLU(alpha=0.2),
        Dense(seq_length * returns.shape[1]),
        Reshape((seq_length, returns.shape[1]))
    ])
    return model


def build_discriminator():
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(seq_length, returns.shape[1])),
        LSTM(32),
        Dense(1, activation='sigmoid')
    ])
    return model


generator = build_generator()
discriminator = build_discriminator()

discriminator.compile(optimizer='adam', loss='binary_crossentropy')

discriminator.trainable = False

gan_input = tf.keras.Input(shape=(100,))
generated_returns = generator(gan_input)
validity = discriminator(generated_returns)

gan = tf.keras.Model(gan_input, validity)
gan.compile(optimizer='adam', loss='binary_crossentropy')

# Train the GAN
batch_size = 32
epochs = 100

for epoch in range(epochs):
    noise = np.random.normal(0, 1, (batch_size, 100))
    generated_sequences = generator.predict(noise)
    
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    real_sequences = X_train[idx]
    
    d_loss_real = discriminator.train_on_batch(real_sequences, np.ones((batch_size, 1)))
    d_loss_fake = discriminator.train_on_batch(generated_sequences, np.zeros((batch_size, 1)))
    d_loss = 0.5 * (d_loss_real + d_loss_fake)
    
    g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, D Loss: {d_loss}, G Loss: {g_loss}")

# Generate synthetic portfolio scenarios
noise = np.random.normal(0, 1, (1, 100))
generated_returns = generator.predict(noise)
simulated_returns = scaler.inverse_transform(generated_returns.reshape(seq_length, returns.shape[1]))

print(simulated_returns)

# Save the model
generator.save('portfolio_management_gan.h5')

# This script sets up a GAN to simulate portfolio return scenarios, aiding in risk assessment and strategy testing.
# Let me know if you’d like me to refine or expand it further!


python code for GANs in market prediction

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Reshape, LeakyReLU
from sklearn.preprocessing import MinMaxScaler

# Load and preprocess data
data = pd.read_csv('market_data.csv')
prices = data['Close'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
prices_scaled = scaler.fit_transform(prices)

# Prepare training data
seq_length = 60
X_train, y_train = [], []
for i in range(len(prices_scaled) - seq_length):
    X_train.append(prices_scaled[i:i + seq_length])
    y_train.append(prices_scaled[i + seq_length])
X_train, y_train = np.array(X_train), np.array(y_train)

# Build the GAN
def build_generator():
    model = Sequential([
        Dense(128, input_dim=100),
        LeakyReLU(alpha=0.2),
        Dense(256),
        LeakyReLU(alpha=0.2),
        Dense(seq_length),
        Reshape((seq_length, 1))
    ])
    return model


def build_discriminator():
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(seq_length, 1)),
        LSTM(32),
        Dense(1, activation='sigmoid')
    ])
    return model


generator = build_generator()
discriminator = build_discriminator()

discriminator.compile(optimizer='adam', loss='binary_crossentropy')

discriminator.trainable = False

gan_input = tf.keras.Input(shape=(100,))
generated_sequence = generator(gan_input)
validity = discriminator(generated_sequence)

gan = tf.keras.Model(gan_input, validity)
gan.compile(optimizer='adam', loss='binary_crossentropy')

# Train the GAN
batch_size = 32
epochs = 100

for epoch in range(epochs):
    noise = np.random.normal(0, 1, (batch_size, 100))
    generated_sequences = generator.predict(noise)
    
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    real_sequences = X_train[idx]
    
    d_loss_real = discriminator.train_on_batch(real_sequences, np.ones((batch_size, 1)))
    d_loss_fake = discriminator.train_on_batch(generated_sequences, np.zeros((batch_size, 1)))
    d_loss = 0.5 * (d_loss_real + d_loss_fake)
    
    g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, D Loss: {d_loss}, G Loss: {g_loss}")

# Generate market predictions
noise = np.random.normal(0, 1, (1, 100))
generated_sequence = generator.predict(noise)
predicted_prices = scaler.inverse_transform(generated_sequence.reshape(-1, 1))

print(predicted_prices)

# Save the model
generator.save('market_predictor_gan.h5')

# This script sets up a basic GAN to predict market prices based on historical data sequences.
# Let me know if you’d like me to refine or expand it further!


In [ ]:
market prediction with diffusion model in GenAI

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

# Bass diffusion model for GenAI adoption
def bass_model(N, t, p, q, M):
    dNdt = (p + q * (N / M)) * (M - N)
    return dNdt

# Parameters for GenAI adoption
M = 10000  # Market potential (total potential users)
p = 0.05   # Coefficient of innovation (early adopters)
q = 0.4    # Coefficient of imitation (network effects)

# Time range
t = np.linspace(0, 50, 500)

# Initial condition (initial adopters)
N0 = 10

# Solve the differential equation
N = odeint(bass_model, N0, t, args=(p, q, M))

# Plotting the adoption curve
plt.figure(figsize=(10, 6))
plt.plot(t, N, label='GenAI Adopters over Time', color='purple')
plt.xlabel('Time (years)')
plt.ylabel('Number of Adopters')
plt.title('Market Prediction for Generative AI Adoption (Bass Model)')
plt.legend()
plt.grid()
plt.show()

# Let me know if you’d like to add more complexity, like external shocks or real-world data integration! 🚀
